In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - apply_kSF_to_esf
# AUTHOR - James Lane
# PROJECT - ges_mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Use the gridded spline interpolation of the kinematic selection function 
to correct the effective selection function
'''

__author__ = "James Lane"

In [ ]:
### Imports

## Basic
import numpy as np, os, sys
from matplotlib import pyplot as plt
from astropy import units as apu
import dill as pickle

## galpy
from galpy import orbit
from galpy import potential

## Project-specific
sys.path.insert(0,'../../src/')
from ges_mass import ssf as pssf
from ges_mass import plot as pplot
from ges_mass import util as putil

In [ ]:
### Notebook setup
%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

## Keywords & Parameters

In [ ]:
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','RO','VO','ZO']
data_dir_base,apogee_dr,apogee_results_vers,gaia_dr,ro,vo,zo\
    = putil.parse_config_dict(cdict,keywords)

# Pathing
gaia_apogee_dir = 'apogee_'+apogee_dr+'_'+apogee_results_vers+'_gaia_'+gaia_dr+'/'
data_dir = data_dir_base+'gaia_apogee/'+gaia_apogee_dir
kSF_dir = data_dir_base+'ksf/'+gaia_apogee_dir
fig_dir = './fig/'

# Filenames
apogee_effSF_filename = data_dir+'apogee_effSF_grid_inclArea.dat'
apogee_fields_filename = data_dir+'apogee_fields.npy'

# Forcing

# Density potential for weighting
denspot = potential.PowerSphericalPotentialwCutoff(alpha=3.5,rc=30*apu.kpc)

# Distance grid
ndistmods=301
minmax_distmods=[7.,19.]
distmods = np.linspace(minmax_distmods[0], minmax_distmods[1], ndistmods)
ds = 10.**(distmods/5-2)

## Load the data

In [ ]:
# Load the effective SF grid
print('APOGEE data release is '+apogee_dr+', and results_version is: '\
      +apogee_results_vers)
print('Loading APOGEE effective selection function from '+apogee_effSF_filename)
with open(apogee_effSF_filename,'rb') as f:
    effSF_grid = pickle.load(f)
##wi

# Load APOGEE field information
print('Loading APOGEE field information from '+apogee_fields_filename)
apogee_fields = np.load(apogee_fields_filename)
nfields = len(apogee_fields)

# Make sure the effective selection function grid is consistent with our 
# distance modulus grid and apogee fields
assert effSF_grid.shape == (nfields,ndistmods), 'effective selection function'\
    +' grid shape does not match number apogee fields x number distances'

## Apply the kSF to the ESSF

In [ ]:
selec_spaces = [['AD',],
                ['eLz',],
                ['JRLz'],
                ['AD','eLz'],
                ['AD','JRLz'],
                ['eLz','JRLz'],
               ]

for i in range(len(selec_spaces)):
    keffSF_grid = pssf.apply_kSF_splines_to_effSF(selec_spaces[i],effSF_grid,
        apogee_fields, ds,kSF_dir,fig_dir,ro,vo,zo,make_SF_plots=True,
        denspot=denspot)
    
    